In [1]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as Soup
from selenium import webdriver

import tqdm.notebook
import pandas as pd

import time
import os

---

### 상위 출생신고 이름 현황

In [24]:
# URL 및 옵션 설정
url = "https://stfamily.scourt.go.kr/st/StFrrStatcsView.do?pgmId=090000000025"
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome("chromedriver.exe", options=options)

# 화면 진입
driver.get(url)
time.sleep(3)

# iframe 탐색 및 이동
iframe = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/iframe')
driver.switch_to.frame(iframe)

# 조회유형 설정
opt_kind = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_selector"]/div/table/tbody/tr/td[2]')
opt_kind.click()
opt_kind_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_selector_elem_RowTR"]')
opt_kind_year.click()
time.sleep(1)

# 시작연도, 종료연도 설정
opt_start_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_styear"]/div/table/tbody/tr/td[2]/span')
opt_end_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_edyear"]/div/table/tbody/tr/td[2]/span')

# 연도 설정. start_year 변수를 통해 시작할 연도 변경 가능
year = 2023
start_year = 2022

# 저장할 폴더 생성
path_data_dir = "data"

if os.path.isdir(path_data_dir) == False:
    os.mkdir(path_data_dir)


try:
    for i in tqdm.notebook.trange(1, 2022-2008+2, desc='연도별'):
        # 연도 저장한 데이터프레임
        df_result = pd.DataFrame()

        # 현재연도 변경
        year -= 1
        
        # 시작연도 선택
        opt_start_year.click()
        time.sleep(2)
        ele_start_year = driver.find_elements(By.XPATH, '//*[@id="param_MultiCandType_styear_elem_RowTR"]')[-i]
        ele_start_year.click()
        
        # 종료연도 선택
        opt_end_year.click()
        time.sleep(2)
        ele_end_year = driver.find_elements(By.XPATH, '//*[@id="param_MultiCandType_edyear_elem_RowTR"]')[-i]
        ele_end_year.click()

        # 시작연도까지 이동하기
        if year > start_year:
            continue

        # 성별 설정
        opt_gender = driver.find_element(By.XPATH, '//*[@id="param_GenderCd"]/div/table/tbody/tr/td[2]/span')
        opt_gender.click()
        time.sleep(2)
        ele_gender = driver.find_element(By.XPATH, '/html/body').find_elements(By.XPATH, '*')[-1].find_element(By.XPATH, 'div[2]/div/table/tbody/tr[3]/td[1]/input')
        ele_gender.click()

        # 시도 옵션 클릭
        opt_sido = driver.find_element(By.XPATH, '//*[@id="param_SidoCd"]/div/table/tbody/tr/td[2]/span')
        opt_sido.click()
        time.sleep(1)
        
        sidos = driver.find_element(By.XPATH, '/html/body').find_elements(By.XPATH, '*')[-4]
        for sido in tqdm.notebook.trange(2, 19, desc="시도별"):
            # 현재 시도 체크
            ele_sido = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]')
            sido_name = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]/td[3]/div').text
            ele_sido.find_element(By.XPATH, 'td[1]/input').click()
            time.sleep(0.5)

            # 조회버튼 클릭
            driver.find_element(By.XPATH, '//*[@id="btn_query"]').click()
            time.sleep(5)

            # 표에 있는 html을 가져오고 이를 저장
            try:
                row = driver.find_element(By.XPATH, '//*[@id="grid_sheet"]/tbody/tr[3]/td/div/div[1]/table/tbody')
                soup = Soup(row.get_attribute('innerHTML'), "html.parser")
                for row in [[td.text for td in row.findAll("td")[1:]] for row in soup.findAll("tr")[1:]]:
                    df_result = pd.concat([df_result, pd.Series({"순위":row[0], "지역":sido_name, "년도":year, "성별":"남성", "이름":row[1], "비율":row[2], "건수":row[3]})], axis=1, ignore_index=True)
            except:
                pass

            # 시도 옵션 클릭
            opt_sido = driver.find_element(By.XPATH, '//*[@id="param_SidoCd"]/div/table/tbody/tr/td[2]/span')
            opt_sido.click()
            time.sleep(1)

            # 현재 시도 체크 해제
            ele_sido = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]')
            ele_sido.find_element(By.XPATH, 'td[1]/input').click()
            time.sleep(0.5)
            
        # 현재 연도 결과를 csv 파일로 저장
        file_name = f"male_top_name_{year}.csv"
        df_result.T.to_csv(os.path.join(path_data_dir, file_name), index=False)
        
except Exception as e:
    print(year, "연도", sido_name, "중 오류 발생")
    print("오류 내용:", e)


연도별:   0%|          | 0/15 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

...

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
df_total = pd.DataFrame()
path_data_dir = "data"

for y in range(2008, 2023):
    file_name = f"male_top_name_{y}.csv"
    
    df_total = pd.concat([df_total, pd.read_csv(os.path.join(path_data_dir, file_name))])
    

df_total.to_csv(os.path.join(path_data_dir, "male_top_name_2008_2022.csv"), index=False)

---

### 상위 개명신고 이름 현황

In [3]:
# URL 및 옵션 설정
url = "https://stfamily.scourt.go.kr/st/StFrrStatcsView.do?pgmId=090000000062"
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome("chromedriver.exe", options=options)

# 화면 진입
driver.get(url)
time.sleep(3)

# iframe 탐색 및 이동
iframe = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/iframe')
driver.switch_to.frame(iframe)

# 조회유형 설정
opt_kind = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_selector"]/div/table/tbody/tr/td[2]')
opt_kind.click()
opt_kind_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_selector_elem_RowTR"]')
opt_kind_year.click()
time.sleep(1)

# 시작연도, 종료연도 설정
opt_start_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_styear"]/div/table/tbody/tr/td[2]/span')
opt_end_year = driver.find_element(By.XPATH, '//*[@id="param_MultiCandType_edyear"]/div/table/tbody/tr/td[2]/span')

# 연도 설정. start_year 변수를 통해 시작할 연도 변경 가능
year = 2023
start_year = 2019

# 저장할 폴더 생성
path_data_dir = "data"

if os.path.isdir(path_data_dir) == False:
    os.mkdir(path_data_dir)
sido_name = "none"

try:
    for i in tqdm.notebook.trange(1, 2022-2011+2, desc='연도별'):
        # 연도 저장한 데이터프레임
        df_result = pd.DataFrame()

        # 현재연도 변경
        year -= 1
        
        # 시작연도 선택
        opt_start_year.click()
        time.sleep(2)
        ele_start_year = driver.find_elements(By.XPATH, '//*[@id="param_MultiCandType_styear_elem_RowTR"]')[-i]
        ele_start_year.click()
        
        # 종료연도 선택
        opt_end_year.click()
        time.sleep(2)
        ele_end_year = driver.find_elements(By.XPATH, '//*[@id="param_MultiCandType_edyear_elem_RowTR"]')[-i]
        ele_end_year.click()

        # 시작연도까지 이동하기
        if year > start_year:
            continue

        # 성별 설정
        opt_gender = driver.find_element(By.XPATH, '//*[@id="param_GenderCd"]/div/table/tbody/tr/td[2]/span')
        opt_gender.click()
        time.sleep(2)
        ele_gender = driver.find_element(By.XPATH, '/html/body').find_elements(By.XPATH, '*')[-1].find_element(By.XPATH, 'div[2]/div/table/tbody/tr[3]/td[1]/input')
        ele_gender.click()

        # 시도 옵션 클릭
        opt_sido = driver.find_element(By.XPATH, '//*[@id="param_SidoCd"]/div/table/tbody/tr/td[2]/span')
        opt_sido.click()
        time.sleep(1)
        
        sidos = driver.find_element(By.XPATH, '/html/body').find_elements(By.XPATH, '*')[-4]
        for sido in tqdm.notebook.trange(2, 19, desc="시도별"):
            # 현재 시도 체크
            ele_sido = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]')
            sido_name = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]/td[3]/div').text
            ele_sido.find_element(By.XPATH, 'td[1]/input').click()
            time.sleep(0.5)

            # 조회버튼 클릭
            driver.find_element(By.XPATH, '//*[@id="btn_query"]').click()
            time.sleep(5)

            # 표에 있는 html을 가져오고 이를 저장
            try:
                row = driver.find_element(By.XPATH, '//*[@id="grid_sheet"]/tbody/tr[3]/td/div/div[1]/table/tbody')
                soup = Soup(row.get_attribute('innerHTML'), "html.parser")
                for row in [[td.text for td in row.findAll("td")[1:]] for row in soup.findAll("tr")[1:]]:
                    df_result = pd.concat([df_result, pd.Series({"순위":row[0], "지역":sido_name, "년도":year, "성별":"남성", "이름":row[1], "비율":row[2], "건수":row[3]})], axis=1, ignore_index=True)
            except:
                pass

            # 시도 옵션 클릭
            opt_sido = driver.find_element(By.XPATH, '//*[@id="param_SidoCd"]/div/table/tbody/tr/td[2]/span')
            opt_sido.click()
            time.sleep(1)

            # 현재 시도 체크 해제
            ele_sido = sidos.find_element(By.XPATH, f'div[2]/div/table/tbody/tr[{sido}]')
            ele_sido.find_element(By.XPATH, 'td[1]/input').click()
            time.sleep(0.5)
            
        # 현재 연도 결과를 csv 파일로 저장
        file_name = f"male_top_change_name_{year}.csv"
        df_result.T.to_csv(os.path.join(path_data_dir, file_name), index=False)
        
except Exception as e:
    print(year, "년", sido_name, "지역 크롤링 중 오류 발생")
    print("오류 내용:", e)


C:\Users\user\AppData\Local\Temp\ipykernel_9204\297749269.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


연도별:   0%|          | 0/12 [00:00<?, ?it/s]

시도별:   0%|          | 0/17 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
df_total = pd.DataFrame()
path_data_dir = "data"

for y in range(2011, 2023):
    file_name = f"male_top_change_name_{y}.csv"
    
    df_total = pd.concat([df_total, pd.read_csv(os.path.join(path_data_dir, file_name))])
    
df_total.to_csv(os.path.join(path_data_dir, "male_top_change_name_2011_2022.csv"), index=False)